# 1. Análises Comparativas

## 1.1. Análises IntraClusters

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import date

class ClusterMetrics:
    """
    Classe para análise estatística de clusters de fundos de investimento.
    Realiza carregamento de dados, preparação de colunas auxiliares e cálculo de métricas agregadas,
    incluindo médias, desvios padrão, proporções e índice de concentração HHI.
    """

    def __init__(self, spark, tipo_fundo: str):
        """
        Inicializa a classe com o objeto Spark e o tipo de fundo.
        :param spark: sessão Spark ativa
        :param tipo_fundo: tipo de fundo a ser analisado (ex: 'fidc', 'fii', 'fip')
        """
        self.spark = spark
        self.tipo_fundo = tipo_fundo.lower()
        self.df = None
        self.final_df = None

    def load_data(self):
        """
        Carrega os dados do cluster para o tipo de fundo especificado.
        Espera-se que a tabela contenha as colunas necessárias para as análises seguintes.
        """
        path = f"desafio_kinea.prospecto_fundos.resultados_cluster_{self.tipo_fundo}"
        self.df = self.spark.table(path)

    def prepare_columns(self):
        """
        Prepara colunas auxiliares para análise, como idade do fundo e flag de chamada de capital IPCA.
        'idade_anos' é calculada como a diferença entre a data atual e a menor data de emissão por CNPJ.
        Cria flag binária para identificar fundos com chamada de capital IPCA.
        """
        w_cnpj = Window.partitionBy("cnpj")
        self.df = self.df.withColumn(
            "idade_anos",
            F.datediff(F.current_date(), F.min("data_emissao").over(w_cnpj)) / 365.25
        )
        self.df = self.df.withColumn(
            "chamada_capital_ipca_flag",
            F.when(
                F.upper(F.coalesce(F.col("chamada_capital_ipca"), F.lit("NÃO"))).like("%SIM%"),
                1
            ).otherwise(0)
        )

    def calculate_aggregates(self):
        """
        Calcula métricas agregadas por cluster, incluindo médias, desvios padrão e índice HHI.
        volume_total_global é o somatório do volume_base_emissao de todos os fundos do tipo analisado.
        Agrega métricas estatísticas por cluster_id_full e calcula o índice de concentração HHI
        para o volume_base_emissao dentro de cada cluster_id_full.
        Junta as métricas agregadas com o HHI e calcula a participação de mercado do cluster.
        """
        volume_total_global = self.df.agg(F.sum("volume_base_emissao").alias("total")).collect()[0]["total"]

        agg_df = self.df.groupBy("cluster_id_full").agg(
            F.countDistinct("cnpj").alias("qtd_fundos"),
            F.round(F.mean("volume_base_emissao"), 2).alias("media_volume_base_emissao"),
            F.round(F.mean("qt_emissoes"), 2).alias("media_qt_emissoes"),
            F.mean("valor_cota_emissao").alias("media_valor_cota_emissao"),
            F.mean("taxa_distribuicao_emissao").alias("media_taxa_distribuicao_emissao"),
            F.mean("quantidade_cotas_totais").alias("media_quantidade_cotas_totais"),
            F.mean("percentual_oferta_institucional").alias("media_percentual_oferta_institucional"),
            F.mean("montante_minimo_emissao").alias("media_montante_minimo_emissao"),
            F.stddev("volume_base_emissao").alias("std_volume_base_emissao"),
            F.stddev("qt_emissoes").alias("std_qt_emissoes"),
            F.stddev("valor_cota_emissao").alias("std_valor_cota_emissao"),
            F.stddev("taxa_distribuicao_emissao").alias("std_taxa_distribuicao_emissao"),
            F.stddev("quantidade_cotas_totais").alias("std_quantidade_cotas_totais"),
            F.stddev("percentual_oferta_institucional").alias("std_percentual_oferta_institucional"),
            F.stddev("montante_minimo_emissao").alias("std_montante_minimo_emissao"),
            F.mean("chamada_capital_ipca_flag").alias("proporcao_chamada_capital_ipca"),
            F.mean("sharpe_ratio").alias("media_sharpe_ratio"),
            F.sum("volume_base_emissao").alias("volume_total_cluster"),
            F.mean("idade_anos").alias("idade_media_anos")
        )

        w_cluster = Window.partitionBy("cluster_id_full")
        df_hhi = self.df.withColumn(
            "prop_volume_cluster",
            F.col("volume_base_emissao") / F.sum("volume_base_emissao").over(w_cluster)
        ).withColumn(
            "hhi_component",
            F.pow(F.col("prop_volume_cluster"), 2)
        )

        hhi_df = df_hhi.groupBy("cluster_id_full").agg(
            F.sum("hhi_component").alias("hhi_volume_base_emissao")
        )

        self.final_df = agg_df.join(hhi_df, on="cluster_id_full", how="left").withColumn(
            "proporcao_participacao_mercado",
            F.col("volume_total_cluster") / F.lit(volume_total_global)
        )

    def run(self):
        """
        Executa o pipeline de análise: carrega dados, prepara colunas e calcula agregados.
        :return: DataFrame final com métricas agregadas por cluster
        """
        self.load_data()
        self.prepare_columns()
        self.calculate_aggregates()
        return self.final_df

In [0]:
tipos_fundo = ["fidc", "fii", "fip"]

# Itera sobre a lista e aplica a classe
resultados = {}
for tipo in tipos_fundo:
    cm = ClusterMetrics(spark, tipo)
    resultados[tipo] = cm.run()

# Visualizando
for tipo in tipos_fundo:
    display(resultados[tipo])

cluster_id_full,qtd_fundos,media_volume_base_emissao,media_qt_emissoes,media_valor_cota_emissao,media_taxa_distribuicao_emissao,media_quantidade_cotas_totais,media_percentual_oferta_institucional,media_montante_minimo_emissao,std_volume_base_emissao,std_qt_emissoes,std_valor_cota_emissao,std_taxa_distribuicao_emissao,std_quantidade_cotas_totais,std_percentual_oferta_institucional,std_montante_minimo_emissao,proporcao_chamada_capital_ipca,media_sharpe_ratio,volume_total_cluster,idade_media_anos,hhi_volume_base_emissao,proporcao_participacao_mercado
FIDC_3,1,1.5E8,1.0,1000.0,null,115000.0,null,1000000.0,null,null,null,null,null,null,null,0.0,null,1.5E8,9.697467488021903,1.0,0.021330138780068644
FIDC_4,3,2.0E8,1.0,17000.0,null,66666.66666666667,null,1683333.3333333333,null,0.0,13856.406460551018,null,115470.05383792515,null,2872317.5892183883,0.0,null,2.0E8,4.5083276294775265,1.0,0.028440185040091524
FIDC_2,2,null,1.67,16733.333333333332,null,1486666.6666666667,null,null,null,1.1547005383792515,28809.778432562325,null,1287491.1002928654,null,null,0.0,null,null,4.546657540497376,null,null
FIDC_1,3,null,1.5,700.0,null,306250.0,null,null,null,0.7071067811865476,519.6152422706632,null,370967.9062937925,null,null,0.0,null,null,6.278804471822952,null,null
FIDC_0,156,6.888972216E7,1.53,15042.994809365879,0.009999999776482582,1878417.7303166564,80.0,4434352.907563025,9.951721906136455E7,1.226362286029718,92383.1853438529,null,1.9174955743888408E7,null,2.8292588082919355E7,0.0,0.496258689935825,6.68230305E9,6.3136138131566675,0.031601224591647464,0.9502296761798398


cluster_id_full,qtd_fundos,media_volume_base_emissao,media_qt_emissoes,media_valor_cota_emissao,media_taxa_distribuicao_emissao,media_quantidade_cotas_totais,media_percentual_oferta_institucional,media_montante_minimo_emissao,std_volume_base_emissao,std_qt_emissoes,std_valor_cota_emissao,std_taxa_distribuicao_emissao,std_quantidade_cotas_totais,std_percentual_oferta_institucional,std_montante_minimo_emissao,proporcao_chamada_capital_ipca,media_sharpe_ratio,volume_total_cluster,idade_media_anos,hhi_volume_base_emissao,proporcao_participacao_mercado
FII_3,2,1.24773888E9,6.5,100.74499893188477,null,1.4788635E7,null,9977380.0,1.0574624652096683E9,6.363961030678928,1.0535875934249053,null,1.3843220374119962E7,null,null,0.0,-0.6096444024110734,2.49547776E9,9.273100616016428,0.6795653409497315,0.3121755385319782
FII_2,3,3.9829995733E8,4.0,3400.1366678873696,null,4561576.0,null,2.9994580333333332E7,2.626569129979304E8,3.0,5715.64940448146,null,3995161.318561242,null,3.4645710658049E7,0.0,-0.3182210783503385,1.194899872E9,0.32306639288158795,0.42997060004251053,0.14947779419736917
FII_0,2,7.2935E7,7.0,100.0,null,729350.0,null,5.047E7,3.1770307678711582E7,1.4142135623730951,0.0,null,317703.07678711583,null,null,0.0,-0.040207926984517764,1.4587E8,17.681040383299113,0.5474363074947882,0.018247826743068095
FII_1,14,2.5765078614E8,4.81,268.32267018953956,null,4605279.125,null,8.0835454E7,4.0233768453633285E8,4.847250079512437,477.01078829921477,null,7953223.587893024,null,1.999109882218033E8,0.0625,-0.6241943668266001,3.607111006E9,11.78798767967146,0.23316422265102843,0.4512369690855012
FII_4,2,2.75235E8,5.5,54.5,null,3.4974574E7,null,3.0235004E7,3.178657113467887E8,0.7071067811865476,64.34671708797582,null,4.874776330409173E7,null,2.861660577776533E7,0.0,-0.06463431958880625,5.5047E8,9.415468856947296,0.8334418095113616,0.06886187144208332


cluster_id_full,qtd_fundos,media_volume_base_emissao,media_qt_emissoes,media_valor_cota_emissao,media_taxa_distribuicao_emissao,media_quantidade_cotas_totais,media_percentual_oferta_institucional,media_montante_minimo_emissao,std_volume_base_emissao,std_qt_emissoes,std_valor_cota_emissao,std_taxa_distribuicao_emissao,std_quantidade_cotas_totais,std_percentual_oferta_institucional,std_montante_minimo_emissao,proporcao_chamada_capital_ipca,media_sharpe_ratio,volume_total_cluster,idade_media_anos,hhi_volume_base_emissao,proporcao_participacao_mercado
FIP_1,1,4.25982368E8,2.0,40190.80859375,null,10599.0,null,20115.499700546265,0.0,0.0,0.0,null,0.0,null,28390.77410559793,0.0,null,8.51964736E8,8.394250513347023,0.5,0.07454869728940498
FIP_0,25,3.1709172286E8,1.12,52527.84060694774,null,5.70389237027027E7,null,8.412211042424242E7,2.5583651046694732E8,0.6963106238227914,174925.0195025671,null,1.6588011205598024E8,null,1.376380368588808E8,0.0,-0.07940753596060118,8.87856824E9,12.250550345006198,0.05813259909415055,0.7768933010005301
FIP_2,1,7.00099968E8,1.0,1.0,null,7.00099968E8,null,100000.0,null,null,null,null,null,null,null,0.0,null,7.00099968E8,17.626283367556468,1.0,0.06126021228507059
FIP_3,6,9.53330323E7,1.43,457.4412744556154,null,7.214719557142857E7,65.0,8.668333333333333E7,1.1977187304871677E8,0.7867957924694432,509.12253743087433,null,1.886735902170432E8,null,1.4152385252434775E8,0.0,null,4.766651615E8,4.767380463479026,0.4525473705924301,0.041709199138811354
FIP_4,3,1.7366666667E8,2.0,683.3333333333334,null,3347666.6666666665,null,1000000.0,2.827725823578611E8,1.7320508075688772,548.4827557301445,null,5761126.13412806,null,0.0,0.0,null,5.21E8,1.7303216974674882,0.922487759771,0.04558859028618292


In [0]:
for tipo in tipos_fundo:
    # Recupera o DataFrame de clusters para o tipo de fundo
    df_analise = resultados[tipo]

    # Salva o DataFrame resultante no schema especificado, sobrescrevendo se já existir
    tabela = f"desafio_kinea.prospecto_fundos.analise_cluster_{tipo.lower()}"
    df_analise.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela)

## 1.2. Análise Kinea versus Cluster

Avaliando performance **conjunta** dos fundos KINEA em comparação aos fundos concorrentes:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

class KineaClusterComparisonFull:
    def __init__(self, df):
        """
        Inicializa a classe com o DataFrame base
        :param df: DataFrame contendo todos os fundos e clusters
        """
        self.df = df
        self.final_comp_df = None

    def filter_kinea_clusters(self):
        """Mantém apenas clusters onde a Kinea está presente"""
        clusters_kinea_ids = (
            self.df.filter(F.upper(F.col("tipo_gestor")) == "KINEA")
                   .select("cluster_id")
                   .distinct()
        )
        self.df = self.df.join(clusters_kinea_ids, on="cluster_id", how="inner")

    def prepare_columns(self):
        """Prepara colunas auxiliares"""
        # Idade do fundo em anos (considera primeira emissão por CNPJ)
        w_cnpj = Window.partitionBy("cnpj")
        self.df = self.df.withColumn(
            "idade_anos",
            F.datediff(F.current_date(), F.min("data_emissao").over(w_cnpj)) / 365.25
        )

        # Flag binária chamada de capital IPCA
        self.df = self.df.withColumn(
            "chamada_capital_ipca_flag",
            F.when(
                F.upper(F.coalesce(F.col("chamada_capital_ipca"), F.lit("NÃO"))).like("%SIM%"),
                1
            ).otherwise(0)
        )

        # Sharpe do fundo
        self.df = self.df.withColumn(
            "sharpe_ratio_calc",
            F.when(F.col("volatilidade_historica") != 0, F.col("retorno_acumulado") / F.col("volatilidade_historica")).otherwise(0)
        )

        # Volume total do cluster
        w_cluster = Window.partitionBy("cluster_id")
        self.df = self.df.withColumn(
            "volume_total_cluster",
            F.sum("volume_base_emissao").over(w_cluster)
        )

        # Market share do fundo
        self.df = self.df.withColumn(
            "market_share_fundo",
            (F.col("volume_base_emissao") / F.col("volume_total_cluster")) * 100
        )

    def aggregate_metrics(self):
        """Agrega métricas por cluster e tipo_gestor"""
        metrics = [
            "retorno_acumulado",
            "volatilidade_historica",
            "sharpe_ratio_calc",
            "volume_base_emissao",
            "qt_emissoes",
            "valor_cota_emissao",
            "taxa_distribuicao_emissao",
            "quantidade_cotas_totais",
            "percentual_oferta_institucional",
            "montante_minimo_emissao",
            "chamada_capital_ipca_flag",
            "idade_anos"
        ]

        agg_exprs = [F.mean(m).alias(m) for m in metrics]
        agg_exprs += [
            F.sum("volume_base_emissao").alias("volume_total_gestor"),
            F.sum("market_share_fundo").alias("market_share_gestor"),
            F.countDistinct("cnpj").alias("qtd_fundos")
        ]

        self.agg_metrics = self.df.groupBy("cluster_id", "tipo_gestor").agg(*agg_exprs)

    def pivot_and_compare(self):
        """Pivot Kinea vs Concorrentes e organiza lado a lado"""
        # Pivot
        pivot_df = self.agg_metrics.groupBy("cluster_id").pivot("tipo_gestor").agg(
            *[F.first(c).alias(c) for c in self.agg_metrics.columns if c not in ["cluster_id", "tipo_gestor"]]
        )

        metrics = [
            "qtd_fundos",
            "retorno_acumulado",
            "volatilidade_historica",
            "sharpe_ratio_calc",
            "volume_base_emissao",
            "qt_emissoes",
            "valor_cota_emissao",
            "taxa_distribuicao_emissao",
            "quantidade_cotas_totais",
            "percentual_oferta_institucional",
            "montante_minimo_emissao",
            "chamada_capital_ipca_flag",
            "idade_anos",
            "volume_total_gestor",
            "market_share_gestor"
        ]

        # Calcula diferenças Kinea vs Concorrente
        for m in metrics:
            pivot_df = pivot_df.withColumn(
                f"dif_{m}",
                F.coalesce(F.col(f"KINEA_{m}"), F.lit(0)) - F.coalesce(F.col(f"CONCORRENTE_{m}"), F.lit(0))
            )

        # Organiza colunas lado a lado
        cols_lado_a_lado = ["cluster_id"]
        for m in metrics:
            cols_lado_a_lado += [f"KINEA_{m}", f"CONCORRENTE_{m}", f"dif_{m}"]

        self.final_comp_df = pivot_df.filter(F.col("KINEA_qtd_fundos").isNotNull()).select(*cols_lado_a_lado)

    def run(self):
        """Executa todo o pipeline"""
        self.filter_kinea_clusters()
        self.prepare_columns()
        self.aggregate_metrics()
        self.pivot_and_compare()
        return self.final_comp_df

In [0]:

tipos_fundo = ["fidc", "fii", "fip"]

# Itera sobre a lista e aplica a classe
resultados = {}
for tipo in tipos_fundo:
    # Carrega o DataFrame do tipo de fundo
    df_fundo = spark.table(f"desafio_kinea.prospecto_fundos.resultados_cluster_{tipo}")
    
    # Aplica a classe
    kcc = KineaClusterComparisonFull(df_fundo)
    resultados[tipo] = kcc.run()

# Visualizando
for tipo in tipos_fundo:
    display(resultados[tipo])

cluster_id,KINEA_qtd_fundos,CONCORRENTE_qtd_fundos,dif_qtd_fundos,KINEA_retorno_acumulado,CONCORRENTE_retorno_acumulado,dif_retorno_acumulado,KINEA_volatilidade_historica,CONCORRENTE_volatilidade_historica,dif_volatilidade_historica,KINEA_sharpe_ratio_calc,CONCORRENTE_sharpe_ratio_calc,dif_sharpe_ratio_calc,KINEA_volume_base_emissao,CONCORRENTE_volume_base_emissao,dif_volume_base_emissao,KINEA_qt_emissoes,CONCORRENTE_qt_emissoes,dif_qt_emissoes,KINEA_valor_cota_emissao,CONCORRENTE_valor_cota_emissao,dif_valor_cota_emissao,KINEA_taxa_distribuicao_emissao,CONCORRENTE_taxa_distribuicao_emissao,dif_taxa_distribuicao_emissao,KINEA_quantidade_cotas_totais,CONCORRENTE_quantidade_cotas_totais,dif_quantidade_cotas_totais,KINEA_percentual_oferta_institucional,CONCORRENTE_percentual_oferta_institucional,dif_percentual_oferta_institucional,KINEA_montante_minimo_emissao,CONCORRENTE_montante_minimo_emissao,dif_montante_minimo_emissao,KINEA_chamada_capital_ipca_flag,CONCORRENTE_chamada_capital_ipca_flag,dif_chamada_capital_ipca_flag,KINEA_idade_anos,CONCORRENTE_idade_anos,dif_idade_anos,KINEA_volume_total_gestor,CONCORRENTE_volume_total_gestor,dif_volume_total_gestor,KINEA_market_share_gestor,CONCORRENTE_market_share_gestor,dif_market_share_gestor
0,2,155,-153,-0.01976334689172865,4.720080042573507,-4.739843389465236,0.17352356017927267,3.8263233844280435,-3.652799824248771,-0.05694715712180672,0.016529605911590427,-0.07347676303339715,4.26269584E8,6.13659356E7,3.649036484E8,3.5,1.5096153846153846,1.9903846153846154,117.41999816894531,15170.563824846195,-15053.14382667725,null,0.009999999776482582,-0.009999999776482582,4693700.0,1857717.125392514,2835982.8746074857,null,80.0,-80.0,2.5072548E7,4081563.247863248,2.0990984752136752E7,0.0,0.0,0.0,4.618754277891854,6.325753915529246,-1.7069996376373915,8.52539168E8,5.829763882E9,-4.977224714E9,12.758163789054734,87.24183621094527,-74.48367242189053


cluster_id,KINEA_qtd_fundos,CONCORRENTE_qtd_fundos,dif_qtd_fundos,KINEA_retorno_acumulado,CONCORRENTE_retorno_acumulado,dif_retorno_acumulado,KINEA_volatilidade_historica,CONCORRENTE_volatilidade_historica,dif_volatilidade_historica,KINEA_sharpe_ratio_calc,CONCORRENTE_sharpe_ratio_calc,dif_sharpe_ratio_calc,KINEA_volume_base_emissao,CONCORRENTE_volume_base_emissao,dif_volume_base_emissao,KINEA_qt_emissoes,CONCORRENTE_qt_emissoes,dif_qt_emissoes,KINEA_valor_cota_emissao,CONCORRENTE_valor_cota_emissao,dif_valor_cota_emissao,KINEA_taxa_distribuicao_emissao,CONCORRENTE_taxa_distribuicao_emissao,dif_taxa_distribuicao_emissao,KINEA_quantidade_cotas_totais,CONCORRENTE_quantidade_cotas_totais,dif_quantidade_cotas_totais,KINEA_percentual_oferta_institucional,CONCORRENTE_percentual_oferta_institucional,dif_percentual_oferta_institucional,KINEA_montante_minimo_emissao,CONCORRENTE_montante_minimo_emissao,dif_montante_minimo_emissao,KINEA_chamada_capital_ipca_flag,CONCORRENTE_chamada_capital_ipca_flag,dif_chamada_capital_ipca_flag,KINEA_idade_anos,CONCORRENTE_idade_anos,dif_idade_anos,KINEA_volume_total_gestor,CONCORRENTE_volume_total_gestor,dif_volume_total_gestor,KINEA_market_share_gestor,CONCORRENTE_market_share_gestor,dif_market_share_gestor
1,5,9,-4,-0.046174580757185946,827.1014489111369,-827.1476234918941,0.16557110743794015,13.06033316436729,-12.89476205692935,-0.1340167922749292,5.892798460578884,-6.0268152528538135,6.508489472E8,3.9207363333333336E7,6.116415838666667E8,4.2,5.090909090909091,-0.8909090909090907,93.41800079345703,355.77500488758085,-262.3570040941238,null,null,0.0,8328711.4,2912809.909090909,5415901.490909091,null,null,0.0,1.395015912E8,7502782.5,1.3199880869999999E8,0.2,0.0,0.2,3.162217659137577,15.708792234459587,-12.54657457532201,3.254244736E9,3.5286627E8,2.901378466E9,90.21748237265088,9.782517627349115,80.43496474530176
2,3,null,3,0.0797156849687517,null,0.0797156849687517,0.11453909928921241,null,0.11453909928921241,0.466002935853244,null,0.466002935853244,3.982999573333333E8,null,3.982999573333333E8,4.0,null,4.0,3400.1366678873696,null,3400.1366678873696,null,null,0.0,4561576.0,null,4561576.0,null,null,0.0,2.9994580333333332E7,null,2.9994580333333332E7,0.0,null,0.0,0.32306639288158795,null,0.32306639288158795,1.194899872E9,null,1.194899872E9,100.0,null,100.0
3,1,1,0,0.1360068811234736,-0.07096194018501245,0.20696882130848604,0.09710710014631813,0.13734968994471658,-0.040242589798398454,1.4005863723511711,-0.516651622683493,1.917237995034664,1.99547776E9,5.0E8,1.49547776E9,11.0,2.0,9.0,101.48999786376953,100.0,1.4899978637695312,null,null,0.0,2.457727E7,5000000.0,1.957727E7,null,null,0.0,9977380.0,null,9977380.0,0.0,0.0,0.0,0.8076659822039699,17.738535249828885,-16.930869267624914,1.99547776E9,5.0E8,1.49547776E9,79.96375651931275,20.03624348068724,59.92751303862551
4,1,1,0,0.06616125038983078,0.13495950079488406,-0.06879825040505327,0.15028195631586924,0.15090618247082915,-6.242261549599115E-4,0.4402474655757751,0.8943271812006267,-0.4540797156248516,5.0E8,5.047E7,4.4953E8,5.0,6.0,-1.0,9.0,100.0,-91.0,null,null,0.0,6.9444448E7,504700.0,6.8939748E7,null,null,0.0,1.0000008E7,5.047E7,-4.0469992E7,0.0,0.0,0.0,1.0157426420260096,17.815195071868583,-16.799452429842574,5.0E8,5.047E7,4.4953E8,90.83147128817193,9.168528711828074,81.66294257634385


cluster_id,KINEA_qtd_fundos,CONCORRENTE_qtd_fundos,dif_qtd_fundos,KINEA_retorno_acumulado,CONCORRENTE_retorno_acumulado,dif_retorno_acumulado,KINEA_volatilidade_historica,CONCORRENTE_volatilidade_historica,dif_volatilidade_historica,KINEA_sharpe_ratio_calc,CONCORRENTE_sharpe_ratio_calc,dif_sharpe_ratio_calc,KINEA_volume_base_emissao,CONCORRENTE_volume_base_emissao,dif_volume_base_emissao,KINEA_qt_emissoes,CONCORRENTE_qt_emissoes,dif_qt_emissoes,KINEA_valor_cota_emissao,CONCORRENTE_valor_cota_emissao,dif_valor_cota_emissao,KINEA_taxa_distribuicao_emissao,CONCORRENTE_taxa_distribuicao_emissao,dif_taxa_distribuicao_emissao,KINEA_quantidade_cotas_totais,CONCORRENTE_quantidade_cotas_totais,dif_quantidade_cotas_totais,KINEA_percentual_oferta_institucional,CONCORRENTE_percentual_oferta_institucional,dif_percentual_oferta_institucional,KINEA_montante_minimo_emissao,CONCORRENTE_montante_minimo_emissao,dif_montante_minimo_emissao,KINEA_chamada_capital_ipca_flag,CONCORRENTE_chamada_capital_ipca_flag,dif_chamada_capital_ipca_flag,KINEA_idade_anos,CONCORRENTE_idade_anos,dif_idade_anos,KINEA_volume_total_gestor,CONCORRENTE_volume_total_gestor,dif_volume_total_gestor,KINEA_market_share_gestor,CONCORRENTE_market_share_gestor,dif_market_share_gestor
0,1,24,-23,0.05440764300397527,null,0.05440764300397527,0.23475950687325822,null,0.23475950687325822,0.23175906155463524,0.0,0.23175906155463524,3.2E8,3.169840088888889E8,3015991.1111111045,1.0,1.125,-0.125,100.0,54166.21062591486,-54066.21062591486,null,null,0.0,4000000.0,5.851222713888889E7,-5.451222713888889E7,null,null,0.0,1.0E7,8.6438426375E7,-7.6438426375E7,0.0,0.0,0.0,0.8323066392881588,12.563997262149213,-11.731690622861054,3.2E8,8.55856824E9,-8.23856824E9,3.604184721567224,96.39581527843278,-92.79163055686556
3,1,5,-4,null,0.0,0.0,null,null,0.0,0.0,0.0,0.0,2.0E8,6.9166290375E7,1.30833709625E8,3.0,1.1666666666666667,1.8333333333333333,1000.0,367.01482019821805,632.9851798017819,null,null,0.0,250000.0,8.41300615E7,-8.38800615E7,null,65.0,-65.0,1.0E7,1.25025E8,-1.15025E8,0.0,0.0,0.0,1.322381930184805,5.341546885694729,-4.0191649555099245,2.0E8,2.766651615E8,-7.66651615E7,41.95817444904666,58.041825550953334,-16.083651101906675


In [0]:
for tipo in tipos_fundo:
    # Recupera o DataFrame de clusters para o tipo de fundo
    df_analise = resultados[tipo]

    # Salva o DataFrame resultante no schema especificado, substituindo se já existir
    tabela = f"desafio_kinea.prospecto_fundos.analise_cluster_kinea_conjunto_{tipo.lower()}"
    df_analise.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela)

Avaliando performance **individual** dos fundos KINEA em comparação aos fundos concorrentes:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

class KineaPerformance:
    """
    Classe para análise de performance individual dos fundos KINEA em relação à média dos clusters.
    Realiza carregamento de dados, preparação de colunas auxiliares, cálculo de métricas operacionais e financeiras,
    comparação das métricas dos fundos KINEA com a média do cluster e cálculo do market share.
    """

    def __init__(self, spark, tipo_fundo: str):
        """
        Inicializa a classe com o objeto Spark e o tipo de fundo.
        :param spark: sessão Spark ativa
        :param tipo_fundo: tipo de fundo a ser analisado (ex: 'fidc', 'fii', 'fip')
        """
        self.spark = spark
        self.tipo_fundo = tipo_fundo.lower()
        self.df = None
        self.result_df = None

    def load_data(self):
        """
        Carrega os dados do cluster para o tipo de fundo especificado.
        """
        path = f"desafio_kinea.prospecto_fundos.resultados_cluster_{self.tipo_fundo}"
        self.df = self.spark.table(path)

    def prepare_columns(self):
        """
        Prepara colunas auxiliares para análise:
        - Idade do fundo em anos (diferença entre hoje e primeira emissão por CNPJ)
        - Flag binária para chamada de capital IPCA
        - Sharpe do fundo (retorno/volatilidade), se as colunas existirem
        """
        w_cnpj = Window.partitionBy("cnpj")
        self.df = self.df.withColumn(
            "idade_anos",
            F.datediff(F.current_date(), F.min("data_emissao").over(w_cnpj)) / 365.25
        )
        self.df = self.df.withColumn(
            "chamada_capital_ipca_flag",
            F.when(
                F.upper(F.coalesce(F.col("chamada_capital_ipca"), F.lit("NÃO"))).like("%SIM%"),
                1
            ).otherwise(0)
        )
        if "retorno_acumulado" in self.df.columns and "volatilidade_historica" in self.df.columns:
            self.df = self.df.withColumn(
                "sharpe_ratio_calc",
                F.when(F.col("volatilidade_historica") != 0, F.col("retorno_acumulado") / F.col("volatilidade_historica")).otherwise(0)
            )
        else:
            self.df = self.df.withColumn("sharpe_ratio_calc", F.lit(0))

    def calculate_performance(self):
        """
        Calcula métricas operacionais e financeiras dos fundos KINEA e compara com a média do cluster.
        - Calcula médias por cluster
        - Junta médias do cluster com fundos KINEA
        - Calcula diferenças KINEA vs média do cluster
        - Calcula market share do fundo
        """
        metrics = [
            "volume_base_emissao",
            "qt_emissoes",
            "valor_cota_emissao",
            "taxa_distribuicao_emissao",
            "quantidade_cotas_totais",
            "percentual_oferta_institucional",
            "montante_minimo_emissao",
            "chamada_capital_ipca_flag",
            "sharpe_ratio_calc",
            "idade_anos",
        ]

        financial_metrics = []
        if "retorno_acumulado" in self.df.columns:
            financial_metrics.append("retorno_acumulado")
        if "volatilidade_historica" in self.df.columns:
            financial_metrics.append("volatilidade_historica")

        all_metrics = metrics + financial_metrics

        cluster_metrics = self.df.groupBy("cluster_id_full").agg(
            *[F.mean(m).alias(f"media_cluster_{m}") for m in all_metrics],
            F.sum("volume_base_emissao").alias("volume_total_cluster")
        )

        kinea_fundos = self.df.filter(F.upper(F.col("tipo_gestor")) == "KINEA")

        joined = kinea_fundos.join(cluster_metrics, on="cluster_id_full", how="left")

        for m in all_metrics:
            joined = joined.withColumn(
                f"{m}_diff",
                F.coalesce(F.col(m), F.lit(0)) - F.coalesce(F.col(f"media_cluster_{m}"), F.lit(0))
            )

        joined = joined.withColumn(
            "market_share_fundo",
            (F.col("volume_base_emissao") / F.col("volume_total_cluster") * 100)
        ).withColumn(
            "dif_market_share",
            F.col("market_share_fundo") - (F.col("media_cluster_volume_base_emissao") / F.col("volume_total_cluster") * 100)
        )

        select_cols = ["cnpj", "nome_fundo", "cluster_id_full"]
        for m in all_metrics:
            select_cols += [m, f"media_cluster_{m}", f"{m}_diff"]
        select_cols += ["volume_total_cluster", "market_share_fundo", "dif_market_share"]

        self.result_df = joined.select(*select_cols)

    def run(self):
        """
        Executa o pipeline de análise: carrega dados, prepara colunas e calcula performance.
        :return: DataFrame final com métricas individuais dos fundos KINEA e comparação com o cluster
        """
        self.load_data()
        self.prepare_columns()
        self.calculate_performance()
        return self.result_df

In [0]:
tipos_fundo = ["fidc", "fii", "fip"]
resultados = {}

for tipo in tipos_fundo:
    # Cria instância da classe KineaPerformance para o tipo de fundo
    kperf = KineaPerformance(spark, tipo)
    # Executa o pipeline e salva o DataFrame no dicionário
    resultados[tipo] = kperf.run()

# Visualizar resultados
for tipo in tipos_fundo:
    print(f"=== {tipo.upper()} ===")
    display(resultados[tipo])


=== FIDC ===


cnpj,nome_fundo,cluster_id_full,volume_base_emissao,media_cluster_volume_base_emissao,volume_base_emissao_diff,qt_emissoes,media_cluster_qt_emissoes,qt_emissoes_diff,valor_cota_emissao,media_cluster_valor_cota_emissao,valor_cota_emissao_diff,taxa_distribuicao_emissao,media_cluster_taxa_distribuicao_emissao,taxa_distribuicao_emissao_diff,quantidade_cotas_totais,media_cluster_quantidade_cotas_totais,quantidade_cotas_totais_diff,percentual_oferta_institucional,media_cluster_percentual_oferta_institucional,percentual_oferta_institucional_diff,montante_minimo_emissao,media_cluster_montante_minimo_emissao,montante_minimo_emissao_diff,chamada_capital_ipca_flag,media_cluster_chamada_capital_ipca_flag,chamada_capital_ipca_flag_diff,sharpe_ratio_calc,media_cluster_sharpe_ratio_calc,sharpe_ratio_calc_diff,idade_anos,media_cluster_idade_anos,idade_anos_diff,retorno_acumulado,media_cluster_retorno_acumulado,retorno_acumulado_diff,volatilidade_historica,media_cluster_volatilidade_historica,volatilidade_historica_diff,volume_total_cluster,market_share_fundo,dif_market_share
26324298000189,KINEA INFRA - FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS INCENTIVADOS DE INVESTIMENTO EM INFRAESTRUTURA RENDA FIXA CRÉDITO PRIVADO,FIDC_0,3.52539168E8,6.888972216494845E7,2.8364944583505154E8,6.0,1.5285714285714285,4.4714285714285715,134.84,15042.994809365879,-14908.154813027988,null,0.009999999776482582,-0.009999999776482582,3137400.0,1878417.7303166564,1258982.2696833436,null,80.0,-80.0,2.0145096E7,4434352.907563025,1.5710743092436975E7,0,0.0,0.0,-0.11389431424361345,0.015993279174120377,-0.12988759341773382,8.971937029431896,6.313613813156671,2.6583232162752246,-0.01976334689172865,3.930106144329301,-3.9498694912210297,0.17352356017927267,3.2175234137199147,-3.043999853540642,6.68230305E9,5.275713558067379,4.244785723015832
60431592000128,FUNDO DE INVESTIMENTO EM DIREITOS CREDITÓRIOS KINEA CRÉDITO ESTRUTURADO RESPONSABILIDADE LIMITADA,FIDC_0,5.0E8,6.888972216494845E7,4.3111027783505154E8,1.0,1.5285714285714285,-0.5285714285714285,100.0,15042.994809365879,-14942.994809365879,null,0.009999999776482582,-0.009999999776482582,6250000.0,1878417.7303166564,4371582.269683343,null,80.0,-80.0,3.0E7,4434352.907563025,2.5565647092436977E7,0,0.0,0.0,0.0,0.015993279174120377,-0.015993279174120377,0.2655715263518138,6.313613813156671,-6.048042286804857,null,3.930106144329301,-3.930106144329301,null,3.2175234137199147,-3.2175234137199147,6.68230305E9,7.482450230987355,6.451522395935808


=== FII ===


cnpj,nome_fundo,cluster_id_full,volume_base_emissao,media_cluster_volume_base_emissao,volume_base_emissao_diff,qt_emissoes,media_cluster_qt_emissoes,qt_emissoes_diff,valor_cota_emissao,media_cluster_valor_cota_emissao,valor_cota_emissao_diff,taxa_distribuicao_emissao,media_cluster_taxa_distribuicao_emissao,taxa_distribuicao_emissao_diff,quantidade_cotas_totais,media_cluster_quantidade_cotas_totais,quantidade_cotas_totais_diff,percentual_oferta_institucional,media_cluster_percentual_oferta_institucional,percentual_oferta_institucional_diff,montante_minimo_emissao,media_cluster_montante_minimo_emissao,montante_minimo_emissao_diff,chamada_capital_ipca_flag,media_cluster_chamada_capital_ipca_flag,chamada_capital_ipca_flag_diff,sharpe_ratio_calc,media_cluster_sharpe_ratio_calc,sharpe_ratio_calc_diff,idade_anos,media_cluster_idade_anos,idade_anos_diff,retorno_acumulado,media_cluster_retorno_acumulado,retorno_acumulado_diff,volatilidade_historica,media_cluster_volatilidade_historica,volatilidade_historica_diff,volume_total_cluster,market_share_fundo,dif_market_share
12005956000165,Kinea Renda Imobiliária Fundo de Investimento Imobiliário - FII,FII_1,9.1499469E8,2.5765078614285713E8,6.573439018571429E8,8.0,4.8125,3.1875,159.55,268.32267018953956,-108.77266713778175,null,null,0.0,7168557.0,4605279.125,2563277.875,null,null,0.0,9999956.0,8.0835454E7,-7.0835498E7,0,0.0625,-0.0625,-0.2559063981311877,4.0094186940620675,-4.2653250921932555,1.3826146475017111,11.78798767967146,-10.405373032169749,-0.04079614176568025,590.7735564848814,-590.8143526266471,0.15941821722162078,9.376115433816045,-9.216697216594424,3.607111006E9,25.366413356229273,18.22355621337213
16706958000132,Kinea Rendimentos Imobiliários Fundo de Investimento Imobiliário - FII,FII_3,1.99547776E9,1.24773888E9,7.4773888E8,11.0,6.5,4.5,101.49,100.74499893188477,0.7449989318847656,null,null,0.0,2.457727E7,1.4788635E7,9788635.0,null,null,0.0,9977380.0,9977380.0,0.0,0,0.0,0.0,1.4005863723511711,0.4419673748338391,0.958618997517332,0.8076659822039699,9.273100616016428,-8.465434633812459,0.1360068811234736,0.03252247046923057,0.10348441065424302,0.09710710014631813,0.11722839504551735,-0.020121294899199227,2.49547776E9,79.96375651931275,29.963756519312753
24960430000113,KINEA ÍNDICES DE PREÇOS FUNDO DE INVESTIMENTO IMOBILIÁRIO - FII,FII_1,1.21925005E9,2.5765078614285713E8,9.615992618571429E8,10.0,4.8125,5.1875,97.54,268.32267018953956,-170.78266927401222,null,null,0.0,1.5E7,4605279.125,1.0394720875E7,null,null,0.0,1.9508E7,8.0835454E7,-6.1327454E7,0,0.0625,-0.0625,0.32531528715626046,4.0094186940620675,-3.684103406905807,9.259411362080767,11.78798767967146,-2.5285763175906926,0.03224947159942437,590.7735564848814,-590.7413070132819,0.09913297306539981,9.376115433816045,-9.276982460750645,3.607111006E9,33.80128989576208,26.65843275290494
30130708000128,Kinea High Yield CRI Fundo de Investimento Imobiliário Responsabilidade Limitada,FII_2,5.9489997E8,3.982999573333333E8,1.966000106666667E8,7.0,4.0,3.0,99.15,3400.1366678873696,-3300.9866663614907,null,null,0.0,7500000.0,4561576.0,2938424.0,null,null,0.0,9983810.0,2.9994580333333332E7,-2.0010770333333332E7,0,0.0,0.0,0.8216015957544651,0.466002935853244,0.35559865990122114,0.09034907597535935,0.32306639288158795,-0.2327173169062286,0.091775684525508,0.0797156849687517,0.012059999556756296,0.1117033912783868,0.11453909928921241,-0.0028357080108256105,1.194899872E9,49.78659567552452,16.453262342191188
35864448000138,KINEA SECURITIES FUNDO DE INVESTIMENTO IMOBILIÁRIO - FII,FII_4,5.0E8,2.75235E8,2.24765E8,5.0,5.5,-0.5,9.0,54.5,-45.5,null,null,0.0,6.9444448E7,3.4974574E7,3.4469874E7,null,null,0.0,1.0000008E7,3.0235004E7,-2.0234996E7,0,0.0,0.0,0.4402474655757751,0.6672873233882008,-0.22703985781242575,1.0157426420260096,9.415468856947296,-8.399726214921287,0.06616125038983078,0.10056037559235742,-0.03439912520252664,0.15028195631586924,0.15059406939334918,-3.121130774799419E-4,5.5047E8,90.83147128817193,40.831471288171926
42502802000140,KINEA

=== FIP ===


cnpj,nome_fundo,cluster_id_full,volume_base_emissao,media_cluster_volume_base_emissao,volume_base_emissao_diff,qt_emissoes,media_cluster_qt_emissoes,qt_emissoes_diff,valor_cota_emissao,media_cluster_valor_cota_emissao,valor_cota_emissao_diff,taxa_distribuicao_emissao,media_cluster_taxa_distribuicao_emissao,taxa_distribuicao_emissao_diff,quantidade_cotas_totais,media_cluster_quantidade_cotas_totais,quantidade_cotas_totais_diff,percentual_oferta_institucional,media_cluster_percentual_oferta_institucional,percentual_oferta_institucional_diff,montante_minimo_emissao,media_cluster_montante_minimo_emissao,montante_minimo_emissao_diff,chamada_capital_ipca_flag,media_cluster_chamada_capital_ipca_flag,chamada_capital_ipca_flag_diff,sharpe_ratio_calc,media_cluster_sharpe_ratio_calc,sharpe_ratio_calc_diff,idade_anos,media_cluster_idade_anos,idade_anos_diff,retorno_acumulado,media_cluster_retorno_acumulado,retorno_acumulado_diff,volatilidade_historica,media_cluster_volatilidade_historica,volatilidade_historica_diff,volume_total_cluster,market_share_fundo,dif_market_share
49723694000184,KINEA EQUITY INFRA I FEEDER INSTITUCIONAL I,FIP_3,2.0E8,9.53330323E7,1.046669677E8,3.0,1.4285714285714286,1.5714285714285714,1000.0,457.4412744556154,542.5587255443845,null,null,0.0,250000.0,7.214719557142857E7,-7.189719557142857E7,null,65.0,-65.0,1.0E7,8.668333333333333E7,-7.668333333333333E7,0,0.0,0.0,0.0,0.0,0.0,1.322381930184805,4.767380463479026,-3.4449985332942212,null,0.0,0.0,null,null,0.0,4.766651615E8,41.95817444904666,21.95817444904666
57473619000130,KINEA ESTRATÉGIA INFRA CDI FUNDO DE INVESTIMENTO EM PARTICIPAÇÕES EM INFRAESTRUTURA RESPONSABILIDADE LIMITADA,FIP_0,3.2E8,3.1709172285714287E8,2908277.1428571343,1.0,1.121212121212121,-0.1212121212121211,100.0,52527.84060694774,-52427.84060694774,null,null,0.0,4000000.0,5.70389237027027E7,-5.30389237027027E7,null,null,0.0,1.0E7,8.412211042424242E7,-7.412211042424242E7,0,0.0,0.0,0.23175906155463524,0.006263758420395547,0.22549530313423968,0.8323066392881588,12.250550345006198,-11.418243705718039,0.05440764300397527,0.05440764300397527,0.0,0.23475950687325822,0.23475950687325822,0.0,8.87856824E9,3.604184721567224,0.03275615013865263


In [0]:
for tipo in tipos_fundo:
    # Recupera o DataFrame de clusters para o tipo de fundo
    df_analise = resultados[tipo]

    # Salva o DataFrame resultante no schema especificado, sobrescrevendo se já existir
    tabela = f"desafio_kinea.prospecto_fundos.analise_cluster_kinea_individual_{tipo.lower()}"
    df_analise.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tabela)

## 1.2.1. Análises Gráficas KINEA

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Carregar os dados
df = spark.table("desafio_kinea.prospecto_fundos.resultados_cluster_fidc").toPandas()

# Filtrar dados da Kinea
kinea = df[df['tipo_gestor'].str.upper() == 'KINEA']

# Variáveis para análise
variaveis = [
    "valor_cota_emissao",
    "quantidade_cotas_totais",
    "montante_minimo_emissao",
    "volume_base_emissao", 
    "taxa_distribuicao_emissao"
]

# Configurações de plot
plt.style.use('seaborn')
sns.set_palette("pastel")

for var in variaveis:
    plt.figure(figsize=(15, 6))
    
    # --- BOXPLOT ---
    plt.subplot(1, 2, 1)
    
    # Boxplot com violino
    sns.violinplot(x=var, data=df, color='lightgray', inner=None)
    sns.boxplot(x=var, data=df, width=0.1, color='white', 
                showcaps=True, fliersize=3)
    
    # Destacar Kinea
    for val in kinea[var]:
        plt.axvline(val, color='red', linestyle='-', linewidth=2, alpha=0.7)
    
    # Estatísticas descritivas
    mean_val = df[var].mean()
    median_val = df[var].median()
    plt.axvline(mean_val, color='green', linestyle='--', linewidth=2, label=f'Média: {mean_val:.2f}')
    plt.axvline(median_val, color='blue', linestyle=':', linewidth=2, label=f'Mediana: {median_val:.2f}')
    
    plt.title(f'Distribuição de {var}\nPosição da Kinea', fontsize=12, pad=20)
    plt.xlabel(var)
    plt.legend()
    
    # --- HISTOGRAMA ---
    plt.subplot(1, 2, 2)
    
    # Histograma com KDE
    sns.histplot(df[var], bins=30, kde=True, color='skyblue', alpha=0.7)
    
    # Linhas para Kinea
    for val in kinea[var]:
        plt.axvline(val, color='red', linestyle='-', linewidth=2, alpha=0.7, label='Kinea')
    
    # Linhas para estatísticas
    plt.axvline(mean_val, color='green', linestyle='--', linewidth=2, label='Média')
    plt.axvline(median_val, color='blue', linestyle=':', linewidth=2, label='Mediana')
    
    # Ajustar escala se necessário
    if df[var].skew() > 1:
        plt.xscale('log')
        plt.xlabel(f'{var} (escala log)')
    
    plt.title(f'Histograma de {var}', fontsize=12, pad=20)
    plt.xlabel(var)
    plt.ylabel('Frequência')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

## 1.3. Potenciais Novas Emissões

In [0]:
tipos_fundo = ["fidc", "fii", "fip"]

for tipo in tipos_fundo:
    df = spark.table(f"desafio_kinea.prospecto_fundos.resultados_cluster_{tipo}")
    fundos_potenciais = df.filter(F.col("potencial_nova_emissao") == True)
    display(fundos_potenciais)

cnpj,data_emissao,Ticker,qt_emissoes,nome_fundo,tipo_fundo,valor_cota_emissao,direito_preferencia_sobras_montante_adicional,taxa_distribuicao_emissao,tabela_ativos_fundo,sumario_experiencia_socios,quantidade_cotas_emissao,quantidade_cotas_adicionais_emissao,publico_alvo,obs_publico_alvo,procuracao_AGE,planilha_custos,ordenar_fatores_risco,montante_minimo_emissao,investimento_minimo_cpf_cnpj,investimento_minimo_inst,investimento_maximo_cpf_cnpj,investimento_maximo_inst,historico_cotacao_bolsa,fator_proporcao_dp,diluicao_economica_novas_emissoes,criterio_rateio,carteira_fundos_kinea_intrag,breve_historico_gestor,percentual_oferta_institucional,volume_base_emissao,chamada_capital_ipca,tipo_gestor,quantidade_cotas_totais,tipo_anbima,volatilidade_historica,liquidez_media,drawdown_max,retorno_acumulado,sharpe_ratio,valor_cotado_atual,potencial_nova_emissao,cluster_id,cluster_id_full


cnpj,data_emissao,Ticker,qt_emissoes,nome_fundo,tipo_fundo,valor_cota_emissao,direito_preferencia_sobras_montante_adicional,taxa_distribuicao_emissao,tabela_ativos_fundo,sumario_experiencia_socios,quantidade_cotas_emissao,quantidade_cotas_adicionais_emissao,publico_alvo,obs_publico_alvo,procuracao_AGE,planilha_custos,ordenar_fatores_risco,montante_minimo_emissao,investimento_minimo_cpf_cnpj,investimento_minimo_inst,investimento_maximo_cpf_cnpj,investimento_maximo_inst,historico_cotacao_bolsa,fator_proporcao_dp,diluicao_economica_novas_emissoes,criterio_rateio,carteira_fundos_kinea_intrag,breve_historico_gestor,percentual_oferta_institucional,volume_base_emissao,chamada_capital_ipca,tipo_gestor,quantidade_cotas_totais,tipo_anbima,volatilidade_historica,liquidez_media,drawdown_max,retorno_acumulado,sharpe_ratio,valor_cotado_atual,potencial_nova_emissao,cluster_id,cluster_id_full
01235622000161,2015-07-13,FTCE11,4.0,OPPORTUNITY FUNDO DE INVESTIMENTO IMOBILIÁRIO,FII,1766.05,Não,null,"imóveis comerciais, residenciais, industriais, terrenos, shopping centers, títulos de renda fixa públicos e privados, cotas de fundos de investimento referenciados, renda fixa e curto prazo","Consultor Especializado MÉTRICA EMPREENDIMENTOS E PARTICIPAÇÕES LTDA., com 30 anos de experiência, coordenado por Jomar Monnerat, diretor com MBA em Finanças Corporativas pela FGV e experiência na KPMG e Grupo Opportunity",null,null,Investidores qualificados,,,Não incorrerá em custos adicionais para o Fundo,"Risco de Liquidez (Maior), Dificuldades Financeiras do Incorporador/Construtor (Maior), Risco relacionado a Fatores Macroeconômicos e Regulatórios (Maior), Riscos de Mercado (Maior), Risco de Crédito (Maior), Outros Riscos Exógenos ao Controle do Administrador (Maior), Risco de Desapropriação (Maior), Risco de Patrimônio Negativo (Maior), Riscos do Setor Imobiliário (Maior), Risco de Alterações nas Leis de Zoneamento (Maior), Riscos Ambientais (Maior), Riscos Relativos às Operações de Aquisição de bens Imóveis (Maior), Risco de Regularidade dos Imóveis (Maior), Riscos dos Contratos de Locação ou Arrendamento e Risco de Vacância (Maior), Risco de Desligamento da Carteira (Maior), Riscos quanto às Despesas de Manutenção dos Imóveis (Maior), Riscos decorrentes de Vícios Ocultos e Contingências Não Identificadas ou Não Identificáveis (Maior), Risco na gestão financeira, contábil e/ou imobiliária de empreendimento investido pelo FUNDO (Maior), Risco de Ocorrência de Casos Fortuitos e Eventos de Força Maior (Maior), Risco de sinistro e de perdas decorrentes de sinistros não cobertos pelos seguros contratados (Maior), Riscos Operacionais (Maior), Riscos de Conflitos de interesse (Maior), Risco de Custos Relativos a Eventuais Reclamações de Terceiros (Maior), Risco de concentração da carteira do FUNDO (Maior), Risco em Função da Dispensa de Registro da Oferta das Cotas do FUNDO (Maior), Risco do Bloqueio à Negociação de Cotas (Maior), Risco de Colocação Parcial da Emissão de Cotas (Maior), Risco de Governança (Maior), Risco de Inexistência de Garantia das Aplicações no FUNDO (Maior)",2.5E7,1000000.0,null,null,null,"Mín 1850 / Méd 1912.89 / Máx 2950 (2011), Mín 2950 / Méd 3106 / Máx 3350 (2012), Mín 3500 / Méd 3563.66 / Máx 3700.01 (2013), Mín 3710 / Méd 3710 / Máx 3710 (2014)",,,,,"BNY Mellon Serviços Financeiros, fundada em 1784, líder global em administração e gestão de ativos, com US$ 28,5 trilhões em ativos sob custódia e US$ 1,7 trilhão em ativos sob gestão até março de 2015",null,2.5E7,,CONCORRENTE,0.0,Multiestratégia Gestão Ativa,0.02800389511121603,29.72289156626506,-0.020245367891120653,0.008465619375751299,-3.993086313562455,2841.75,true,1,FII_1
03683056000186,2011-11-01,RCRB11,4.0,FUNDO DE INVESTIMENTO IMOBILIÁRIO RIO BRAVO RENDA CORPORATIVA,FII,1.7,Sim,null,"lajes corporativas, conjuntos comerciais, vagas de garagem, direitos sobre empreendimentos imobiliários","Mario Fleck (CEO), Gustavo Franco (Presidente do Conselho), Paulo Bilyk (CIO), Fabio Ohara Ish

cnpj,data_emissao,Ticker,qt_emissoes,nome_fundo,tipo_fundo,valor_cota_emissao,direito_preferencia_sobras_montante_adicional,taxa_distribuicao_emissao,tabela_ativos_fundo,sumario_experiencia_socios,quantidade_cotas_emissao,quantidade_cotas_adicionais_emissao,publico_alvo,obs_publico_alvo,procuracao_AGE,planilha_custos,ordenar_fatores_risco,montante_minimo_emissao,investimento_minimo_cpf_cnpj,investimento_minimo_inst,investimento_maximo_cpf_cnpj,investimento_maximo_inst,historico_cotacao_bolsa,fator_proporcao_dp,diluicao_economica_novas_emissoes,criterio_rateio,carteira_fundos_kinea_intrag,breve_historico_gestor,percentual_oferta_institucional,volume_base_emissao,chamada_capital_ipca,tipo_gestor,quantidade_cotas_totais,tipo_anbima,volatilidade_historica,liquidez_media,drawdown_max,retorno_acumulado,sharpe_ratio,valor_cotado_atual,potencial_nova_emissao,cluster_id,cluster_id_full
57473619000130,2024-10-19,KNDI11,1.0,KINEA ESTRATÉGIA INFRA CDI FUNDO DE INVESTIMENTO EM PARTICIPAÇÕES EM INFRAESTRUTURA RESPONSABILIDADE LIMITADA,FIP,100.0,Não,null,"Geração Solar, Geração Eólica, Rodovias, Saneamento, Energia, Energia, Geração Eólica, Saneamento, Título Público","Roberta Anchieta da Silva possui mais de 20 anos de experiência no mercado financeiro, atuando na Asset Management e na administração fiduciária do maior banco privado da América Latina. Marcio Verri, CFA e CAIA, sócio, fundador e CEO da Kinea Investimentos. Aymar Almeida, sócio executivo e gestor responsável pela estratégia de investimentos em Infraestrutura. Fábio Massao Inocima, gestor responsável pela estratégia de investimentos em Infraestrutura. Bruno Signorelli, integra a área de gestão de fundos da Kinea desde 2018. Gustavo Mendonça Faria, ingressou na Kinea em agosto de 2022. Pablo Nabil Bou Assi, integra a área de gestão de fundos de investimento em infraestrutura desde fevereiro de 2022. Guilherme Torres, ingressou na Kinea em 2023. Maria Eduarda Mochinski, ingressou na Kinea em 2023. Fernanda Guerra, ingressou na Kinea em 2024. Alessandro Lopes, responsável pelas áreas de Riscos, Controles Internos e Compliance e Controladoria. Ricardo Sakai, responsável pela área de Riscos. Ivan Freitas Simão, integra a área de Relações com Investidores - Fundos Imobiliários da Kinea. Luis Filipe de Callis, ingressou na Kinea em 2021 para atuar na equipe de RI dos fundos listados. Artur Pereira Batista Silva, integra a área de relacionamento com investidores dos fundos listados da Kinea.",3200000.0,800000.0,Investidores qualificados,"Clientes correntistas dos segmentos Institucionais, Corporate, Íon, Private, Personnalité, Uniclass e Agências do Itaú Unibanco",Modelo de procuração incluso,"Taxa de distribuição total 2.05%, comissão de coordenação 0.10%, comissão de distribuição 1.70%, advogados 0.08%, tributos advogados 0.00%, taxa registro CVM 0.04%, taxa registro Anbima 0.01%, taxa B3 distribuição padrão fixa 0.05%, taxa B3 distribuição padrão variável 0.04%, taxa B3 análise oferta 0.00%, taxa B3 listagem 0.00%, anúncio 0.01%, cartório 0.01%, outras despesas 0.01%.","Risco de Crédito (Maior), Desempenho das Emissoras (Maior), Capacidade de Pagamento das Emissoras (Maior), Cobrança dos Ativos e dos Ativos de Liquidez (Maior), Setor de atuação das Emissoras (Maior), Atraso na entrega dos projetos das Emissoras (Maior), Não existência de garantia de eliminação de riscos (Maior), Performance operacional, operação e manutenção (Maior), Concentração do Setor de Atuação das Emissoras (Maior), Concentração e Iliquidez da Carteira do Fundo (Maior), Risco regulatório/jurídico (Maior), Legislação do setor de atuação das Emissoras (Maior), Risco Relacionado as Garantias dos Ativos e/ou Ativos de Liquidez e ao seu Aperfeiçoamento (Maior), Aspectos Tributários (Maior), Riscos de Mercado e de Liquidez da Carteira do FIP-IE (Maior), Risco relativo à concentração e pulverização (Maior), Liquidez Reduzida Das Cotas (Média), Riscos Relativos à Rentabilidade do Investimento (Média), Perda de Benefí